In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle  ##################################
   #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  



import plotly.plotly as py
from plotly.graph_objs import *

# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



##### for getting geolocation data  and to calculate distance between two geolocations
import requests
import json
import geopy.distance   




In [ ]:
# Urban-Rural codes:

# Metropolitan Counties*	
# 1	In large metro area of 1+ million residents
# 2	In small metro area of less than 1 million residents

# Nonmetropolitan Counties	
# 3	Micropolitan area adjacent to large metro area
# 4	Noncore adjacent to large metro area
# 5	Micropolitan area adjacent to small metro area
# 6	Noncore adjacent to small metro area and contains a town of at least 2,500 residents
# 7	Noncore adjacent to small metro area and does not contain a town of at least 2,500 residents
# 8	Micropolitan area not adjacent to a metro area
# 9	Noncore adjacent to micro area and contains a town of at least 2,500 residents
# 10	Noncore adjacent to micro area and does not contain a town of at least 2,500 residents
# 11	Noncore not adjacent to metro or micro area and contains a town of at least 2,500 residents
# 12	Noncore not adjacent to metro or micro area and does not contain a town of at least 2,500 residents


In [2]:
path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'Dropbox_datafile_may22_2017_modified_added_univ_country_geolocation_RUCC_only_US_YAY.csv'
df_original = pd.read_csv(path+input_file, sep=';',na_values=["NAN","-1","null"],low_memory=False, parse_dates=['folder_creation_date','date_last_change']) # set header=0 if i wanna pass it my own list of header names
df_original = df_original.drop('Unnamed: 0', 1)
df_original.shape



(1308727, 26)

In [8]:
df_original.columns

Index([u'folder_id', u'num_folder_members', u'folder_creation_date',
       u'date_last_change', u'user_id', u'email_domain', u'num_adds',
       u'num_edits', u'num_deletes', u'major_content_type',
       u'major_content_ext', u'group_total_publ', u'group_num_papers_last',
       u'group_num_citations', u'folder_lifespan', u'simplified_domain',
       u'University_Name', u'Cleaned_university_name', u'Country',
       u'Geolocation', u'zip', u'lat', u'long', u'Rural_Urban_Continuum_Code',
       u'State_Name', u'Description'],
      dtype='object')

In [4]:
df_original[df_original["Cleaned_university_name"]=="northwestern university"][["folder_id","lat","long",'Cleaned_university_name', 'zip','State_Name', 'Rural_Urban_Continuum_Code','Description', 'Country']].head()

,folder_id,lat,long,Cleaned_university_name,zip,State_Name,Rural_Urban_Continuum_Code,Description,Country
190,2079308,42.056459,-87.675267,northwestern university,60208.0,Illinois,1.0,Metropolitan Counties – In large metro area of...,United States
350,2763620,42.056459,-87.675267,northwestern university,60208.0,Illinois,1.0,Metropolitan Counties – In large metro area of...,United States
659,3501736,42.056459,-87.675267,northwestern university,60208.0,Illinois,1.0,Metropolitan Counties – In large metro area of...,United States
813,4101042,42.056459,-87.675267,northwestern university,60208.0,Illinois,1.0,Metropolitan Counties – In large metro area of...,United States
814,4101042,42.056459,-87.675267,northwestern university,60208.0,Illinois,1.0,Metropolitan Counties – In large metro area of...,United States


In [ ]:
#  48: {'University': 'university of miami',
#   'common_num_citations': nan,
#   'common_num_papers_last': nan,
#   'common_num_publ': nan,
#   'lat': 25.7191685,
#   'log_tot_activity': 0.0,
#   'long': -80.27712530000001,
#   'num_adds': 0.0,
#   'num_deletes': 0.0,
#   'num_edits': 0.0,
#   'ru2003': 1.0,
#   'tot_activity': 0.0,
#   'zip': 33146.0},

In [ ]:
# df_original=df_original.drop_duplicates(subset=["folder_id","user_id"])  # i drop any duplicates by those two indeces
# print df_original.shape
# print df_original.dtypes
# df_original.head()

In [3]:
path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/"
filename="dict_lat_log_aggregate_activity.pickle"


file = open(path+filename,'r')
dict_lat_log_aggregate_activity = pickle.load(file)

df = pd.DataFrame.from_dict(dict_lat_log_aggregate_activity,orient='index')

df=df.dropna(axis=0, how='all')
df.head()

,,Rural_Urban_Continuum_Code,common_num_citations,zip,University,tot_activity,num_deletes,long,log_tot_activity,num_edits,common_num_papers_last,num_adds,lat,common_num_publ
3.159203,101.701442,10.0,NaN,50250.0,universiti kuala lumpur,924.0,0.0,101.701442,6.828712,784.0,NaN,140.0,3.159203,NaN
11.999941,-61.773896,NaN,NaN,NaN,st georges university,1652.0,532.0,-61.773896,7.409742,168.0,NaN,952.0,11.999941,NaN
19.631841,-155.986365,4.0,NaN,96740.0,university of the nations,0.0,0.0,-155.986365,0.000000,0.0,NaN,0.0,19.631841,NaN
21.290583,-157.807398,1.0,NaN,96816.0,chaminade university of honolulu,79644.0,25640.0,-157.807398,11.285322,13989.0,NaN,40015.0,21.290583,NaN
21.296939,-157.817112,1.0,30.0-229.0,96822.0,university of hawaii,36885022.0,8783136.0,-157.817112,17.423316,10233200.0,5.0-18.0,17868686.0,21.296939,8.0-22.0


In [5]:
def get_size (RUCC):   
           
    if math.isnan(RUCC):        
        return 0
    else:
        size=10.-RUCC  # index=1 is the largest city, so i want big circles for low indeces
        return size
    
    
df['size'] = df.Rural_Urban_Continuum_Code.apply(get_size)

In [8]:
#df[df["University"]=="university of miami"]



In [13]:
### add new column from list

# aux_serie = pd.Series(list_sizes)

# df['new_col_URCodes'] = aux_serie.values

In [6]:
df_selection=df[['Rural_Urban_Continuum_Code']].dropna(how='any')  # by default, dropna drops a row if ANY  of the indicated fields are missing (i can also set it to if ALL the fields are missing: how='all')

print df.shape
print df_selection.shape

(1058, 14)
(1036, 1)


In [8]:


scl = [0,"rgb(150,0,90)"],[10000,"rgb(0, 0, 200)"],[80000,"rgb(0, 25, 255)"],\
[100000,"rgb(0, 152, 255)"],[200000,"rgb(44, 255, 150)"],[500000,"rgb(151, 255, 0)"],\
[1000000,"rgb(255, 234, 0)"],[100000000,"rgb(255, 111, 0)"],[300000000,"rgb(255, 0, 0)"]

# list of color scales:
# ['Blackbody',
# 'Bluered',
# 'Blues',
# 'Earth',
# 'Electric',
# 'Greens',
# 'Greys',
# 'Hot',
# 'Jet',
# 'Picnic',
# 'Portland',
# 'Rainbow',
# 'RdBu',
# 'Reds',
# 'Viridis',
# 'YlGnBu',
# 'YlOrRd']



data = [ dict(
    lat = df['lat'],
    lon = df['long'],
    text = df['University'].astype(str)+'<br>zip:'+df['zip'].astype(str)+'<br>'+df['tot_activity'].astype(str) + ' events'+'<br>Urban-Rural:'+df['Rural_Urban_Continuum_Code'].astype(str) ,
    marker = dict(
        color = df['log_tot_activity'],
        colorscale = "Portland",#scl, #'Rainbow', #scl,
        reversescale = False,   # so read is low and blue is high
        opacity = 0.1,
        size = df['size'],#6,
       # symbol =2,
        colorbar = dict(
            #thickness = 10,
            titleside = "right",
            outlinecolor = "rgba(68, 68, 68, 0)",
            ticks = "outside",
            #ticklen = 3,
            title = "level of Activity",
#             showticksuffix = "last",
#             ticksuffix = " edits",
            #dtick = 0.1
        ),
    ),
    type = 'scattergeo'
) ]

layout = dict(
    geo = dict(
        scope = 'north america',  # "world" | "usa" | "europe" | "asia" | "africa" | "north america" | "south america" )  
        showland = True,  # so i should the land contour (not just the location of the datapoints)
        landcolor = "rgb(212, 212, 212)",
        subunitcolor = "rgb(255, 255, 255)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = True,
        lakecolor = "rgb(255, 255, 255)",   # white
        showsubunits = True,  #show state lines
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation = dict(
                lon = -100
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title = 'Activity (aggregated at university level).<br>Circle size proportional to population density at university location.<br> Scroll to zoom in/out',
)


fig = { 'data':data, 'layout':layout }
iplot(fig, filename='activity')
offline.plot(fig, filename='/home/juliaponcela/map_activity.html')


'file:///home/juliaponcela/map_activity.html'

In [ ]:




# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2015_06_30_precipitation.csv')

# scl = [0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],\
# [0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],\
# [0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

# data = [ dict(
#     lat = df['Lat'],
#     lon = df['Lon'],
#     text = df['Globvalue'].astype(str) + ' inches',
#     marker = dict(
#         color = df['Globvalue'],
#         colorscale = scl,
#         reversescale = True,   # so read is low and blue is high
#         opacity = 0.1,
#         size = 3,
#         colorbar = dict(
#             thickness = 10,
#             titleside = "right",
#             outlinecolor = "rgba(68, 68, 68, 0)",
#             ticks = "outside",
#             ticklen = 3,
#             showticksuffix = "last",
#             ticksuffix = " inches",
#             dtick = 0.1
#         ),
#     ),
#     type = 'scattergeo'
# ) ]

# layout = dict(
#     geo = dict(
#         scope = 'north america',  # "world" | "usa" | "europe" | "asia" | "africa" | "north america" | "south america" )  
#         showland = True,  # so i should the land contour (not just the location of the datapoints)
#         landcolor = "rgb(212, 212, 212)",
#         subunitcolor = "rgb(255, 255, 255)",
#         countrycolor = "rgb(255, 255, 255)",
#         showlakes = True,
#         lakecolor = "rgb(255, 255, 255)",
#         showsubunits = True,  #show state lines
#         showcountries = True,
#         resolution = 50,
#         projection = dict(
#             type = 'conic conformal',
#             rotation = dict(
#                 lon = -100
#             )
#         ),
#         lonaxis = dict(
#             showgrid = True,
#             gridwidth = 0.5,
#             range= [ -140.0, -55.0 ],
#             dtick = 5
#         ),
#         lataxis = dict (
#             showgrid = True,
#             gridwidth = 0.5,
#             range= [ 20.0, 60.0 ],
#             dtick = 5
#         )
#     ),
#     title = 'US Precipitation 06-30-2015<br>Source: <a href="http://water.weather.gov/precip/">NOAA</a>',
# )


# fig = { 'data':data, 'layout':layout }
# iplot(fig, filename='precipitation')
# offline.plot(fig, filename='/home/juliaponcela/map_precipitation.html')
